In [6]:
!pip install awswrangler --q

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.24 which is incompatible.


In [2]:
import boto3
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import awswrangler as wr

import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn-colorblind')

### set-up

In [3]:
session = boto3.Session()
s3 = session.client('s3')

In [4]:
bucket_name = "cdo-ililapse-364524684987-bucket"
file_path = "x266754/lapse/"

### pfmc

In [5]:
file_name = "x266754/lapse/pfmc_jan_jun.parquet"

In [6]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
pfmc = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 40 s, sys: 13 s, total: 52.9 s
Wall time: 37 s


In [7]:
pfmc[pfmc['agmt_pkge_id']=='V26808080000'][['policy_id', 'pfmc_cur_month']]

,policy_id,pfmc_cur_month
723939,V2680808,2022-04
1445749,V2680808,2022-02
2456624,V2680808,2022-01
3361473,V2680808,2022-05
4262881,V2680808,2022-03


### eciw

In [8]:
file_name = "x266754/lapse/eciw_jan_jun.parquet"

In [9]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
eciw = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 8.08 s, sys: 3.58 s, total: 11.7 s
Wall time: 19.5 s


In [10]:
eciw['eciw_cur_month'] = eciw['as_of_hist_dt'].astype('datetime64[ns]').map(lambda x: x.strftime('%Y-%m'))
eciw['eciw_prev_month'] =  ((eciw['as_of_hist_dt'] - pd.DateOffset(months=1)).dt.to_period("M")).astype(str)

In [11]:
eciw[eciw['agmt_pkge_id']=='V26808080000'][['agmt_pkge_id', 'policy_id', 'eciw_cur_month']]

,agmt_pkge_id,policy_id,eciw_cur_month
531232,V26808080000,V2680808,2022-03
1044350,V26808080000,V2680808,2022-05
1946577,V26808080000,V2680808,2022-01
2749451,V26808080000,V2680808,2022-04
4408009,V26808080000,V2680808,2022-02


### merge pfmc and eciw 

In [12]:
%%time
df=pfmc.merge(eciw, how='left', 
              left_on=['agmt_pkge_id','pfmc_cur_month'], 
              right_on = ['agmt_pkge_id','eciw_cur_month'],
              suffixes=('', '_remove'))
df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

CPU times: user 3min 11s, sys: 1min 24s, total: 4min 35s
Wall time: 4min 35s


In [13]:
df[df['agmt_pkge_id']=='V26808080000'][['agmt_pkge_id', 'policy_id', 'pfmc_cur_month', 'eciw_cur_month']]

,agmt_pkge_id,policy_id,pfmc_cur_month,eciw_cur_month
727368,V26808080000,V2680808,2022-04,2022-04
1452663,V26808080000,V2680808,2022-02,2022-02
2468476,V26808080000,V2680808,2022-01,2022-01
3377707,V26808080000,V2680808,2022-05,2022-05
4283430,V26808080000,V2680808,2022-03,2022-03


In [14]:
del pfmc, eciw

### clarify

In [15]:
file_name = "x266754/lapse/clarify_jan_jun.parquet"

In [16]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
clarify = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 813 ms, sys: 559 ms, total: 1.37 s
Wall time: 2.1 s


In [17]:
clarify['clarify_cur_month'] = clarify['SR_CREATE_DT'].astype('datetime64[ns]').map(lambda x: x.strftime('%Y-%m'))

In [18]:
clarify['SR_TYPE_CATG'] = np.where(clarify['SR_TYPE_CATG'].isnull(), 'None', clarify['SR_TYPE_CATG'])

In [19]:
clarify.head()

,POLICY_NBR,sr_create_month,num_sr_catg,max_sr_time,SR_TYPE_CATG,SR_CREATE_DT,SR_CLOSE_DT,closed,time_to_close,clarify_cur_month
0,-,2022-09,1,0.0,Corres,2022-09-09,2022-09-09,1,0.0,2022-09
1,000000,2022-02,1,120.0,Corres,2022-02-08,2022-06-08,1,120.0,2022-02
2,000000,2022-05,1,0.0,DeathMasterFileMatch,2022-05-05,2022-05-05,1,0.0,2022-05
3,0000000,2022-03,1,0.0,Corres,2022-03-03,2022-03-03,1,0.0,2022-03
4,0000000,2022-07,1,50.0,DeathMasterFileMatch,2022-07-19,2022-09-07,1,50.0,2022-07


### merge with clarify

In [20]:
%%time
df=df.merge(clarify, how='left', 
              left_on=['policy_id','pfmc_cur_month'], 
              right_on = ['POLICY_NBR','clarify_cur_month'],
              suffixes=('', '_remove'))
df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

CPU times: user 3min 38s, sys: 1min 39s, total: 5min 18s
Wall time: 5min 18s


In [21]:
del clarify

In [22]:
#del df
print(df.shape)

(4957025, 269)


In [25]:
pd.set_option('max_columns', None)
df.head()

,year,month,agmt_pkge_id,vul,ul,term,aos_status_code,val_inforce_cls,kind_code,sub_kind,conv_undrwrt_code,conv_type,spec_id,premium_basis,value_basis,series,band_code,lim_db_guar_per,admin_source_cd,tax_basis,non_forf_prv,div_option,liv_ben_sig,spec_und_code,pension_code,medical_issue_cd,prem_mode,debit_indicator,rptg_country,legal_ent,branch_of_business,pgls_product_code,closed_block_stat,int_loan_typ_cd,abbr_pay_typ_cd,opr_stat_cd,wsc_ind,iss_office_inforce,cur_office_inforce,iss_res_place_id,cur_res_place_id,cur_rmo,iss_prem_mode,iss_offc_det_cd,crnt_offc_det_cd,fgn_ind,policy_part,cvrg_typ,prod_type,pln_id,gnrc_pln_id,ins_nature_cd,pln_desc,has_sps_ind,has_chld_ind,prem_pattern_cd,db_pattern_cd,liv_num,frst_scnd_dth_ind,mod_prem_num,lim_prem_sig_ind,term_cvrg_sig_ind,prem_pay_dur,prem_pay_age_num,cvrg_dur,cvrg_age_num,conv_sig_ind,conv_dur,conv_age_num,plan_type,qual_code,gi_ru_code,unisex_ind,ins_type_code,trad_intsens_code,conv_pct,lvl_prem_prd_num,enhnc_comp_ind,sngl_pay_ind,lvl_prem_prd_age,ecv_ind,lp_cntr_cd,lp_cntr_rsn_cd,lp_bill_cd,lp_bill_rsn_cd,lp_bnf_stat_cd,lp_bnf_rsn_cd,lp_canc_rsn_cd,nature,issue_age,extra_ratings,imp_rating,occ_rating,smoking_habit,pref_rating,select_rating,sex,undw_class,place_name,place_abbrev,acord_state_id,acord_ctry_id,base_face_amt,aip_face_amt,pua_face_amt,puar_face_amt,oyt_face_amt,layer_face_amt,designer_face_amt,rdr1_face_amt,rdr2_face_amt,rdr3_face_amt,rdr4_face_amt,rdr_other_face_amt,mpt_tot_act_val,mpt_3,mpt_4,mpt_1600,mpt_502,mpt_600,mpt_620,rtain_amt,ceded_amt,net_rsk_amt,units,trgt_7_pay,trgt_lmtd_pay,crnt_defcy_guar_fnd,tot_loan_prncpl,tot_loan_fnd_val,unpd_loan_int_due,annlzd_sch_prem,annlzd_bld_prem,modal_prem_sch,modal_prem_bld,grs_ropt_rdrprem_pd,base_pol_count,major_lob,lob_group,product_group,bar_feature,ny_state_specific,ttr_feature,issue_age_2nd,extra_ratings_2nd,imp_rating_2nd,occ_rating_2nd,smoking_habit_2nd,pref_rating_2nd,select_rating_2nd,gender_2nd,undw_class_2nd,base_pol_units,min_cntrct_fund,fund_lvl,holding_co_cd,policy_id,layer_units,rdr_units,layer_count,rdr_count,accum_actual_prem_paid,loan_rate,monthly_coi,prod_vers_desc,mktg_grp_desc,lnb_ind,rdr_term_ill_ind,base_fam_spo_ind,base_fam_child_ind,adb_pri_base_ind,adb_spo_base_ind,adb_layer_ind,aviation_xtra_base_ind,awp_base_ind,opai_ind,extc_ind,upb_ind,temp_xtra_base_ind,wp_base_ind,layer_ind,rdr_oyt_dsgn_ind,rdr_paidup_inc_dsgn_ind,oyt_ind,pua_base_pri_ind,pua_base_spo_ind,pua_rdr_ind,rdr_bar_ind,wp_rdr_child_ind,wp_rdr_pri_ind,wp_rdr_sps_ind,rdr_child_ind,rdr_pri_dec_term_ind,rdr_spo_dec_term_ind,rdr_pri_term_prot_ind,rdr_spo_term_prot_ind,rdr_est_prot_ind,rdr_trgt_term_ind,rdr_5rc_ind,rdr_other_ind,attained_age,attained_age_2nd,enhanced_val,conv_type_detail,policy_id_2nd,val_dt,last_up_dt,prod_issue_dt,issue_dt,reinstmt_dt,lapse_dt,xi_expiry_dt,paid_to_dt,run_dt,var_index_acct_rdr_ind,mpt_10,mpt_11,mpt_20881,run_dt_part,value_basis.1,premium_basis.1,prod_line_desc,prod_desc,prod_vers_desc.1,dtl_prod_desc,acord_prod_typ_cd,acord_prod_typ_desc,full_prod_desc,prod_grp,mktg_grp_desc.1,start_dt,end_dt,pfmc_cur_month,evnt_desc,lapse_ind,surrender_ind,reinstate_ind,evnt_3_month_ago,evnt_cur_month,household_id,subject_id,as_of_hist_dt,subj_acq_dt,mrtl_stat_cd,gndr_cd,subj_state_cd,subj_act_ind,pref_lang_cd,deceased_ind,rtl_dstrb_orphn_cd,ixi_buoyancy,ixi_fin_cohort_cd,subj_aum_amt,hhld_acq_dt,most_rcnt_pur_dt,incm_rng_cd,prim_occup_cd,prmr_prsnc_chld_ind,net_wrth_gold_cd,hhld_qdrnt_cd,wc_tot_asset_amt,wc_asset_mix_anty_amt,wc_asset_mix_bond_amt,wc_asset_mix_dep_amt,wc_asset_mix_mf_amt,wc_asset_mix_oth_asset_amt,wc_asset_mix_stk_amt,eciw_cur_month,eciw_prev_month,POLICY_NBR,sr_create_month,num_sr_catg,max_sr_time,SR_TYPE_CATG,SR_CREATE_DT,SR_CLOSE_DT,closed,time_to_close,clarify_cur_month
0,2022,4,V26680770000,YES,NaN,NaN,A,PP,CVL17,B,None,None,17093,1,30,E,None,37,V,0.0,0.0,0,N,A,0.0,B,A,N,US,AZ,None,030,0,None,0.0,None,N,KADX,KADX,5,5,7.0,A,I,I,N,BASE,PRI,LIFE,VUL II,VUL,PRI,VUL2 CVAT Var DB,0,0,LEVEL,VAR LIF

### Export back

In [26]:
%%time
import awswrangler as wr

wr.s3.to_parquet(
    df=df2,
    path='s3://cdo-ililapse-364524684987-bucket/x266754/lapse/lapse_curated_jan_jun.parquet'
    #dataset=True
)

CPU times: user 2min 2s, sys: 21.5 s, total: 2min 24s
Wall time: 1min 55s


{'paths': ['s3://cdo-ililapse-364524684987-bucket/x266754/lapse/lapse_curated_jan_jun.parquet'],
 'partitions_values': {}}